#  Instruction

In this notebook, we will learn how to implement DQN using Tensorflow for the [Cartpole environment in OpenAI gym](https://gymnasium.farama.org/environments/classic_control/cart_pole/). You are given a basic skeleton but you need to complete the code where appropriate to solve the cartpole problem.

You are free to tweak the code at any part. Your are also free to tweak the hyper-parameters to improve the performance of the agent. At the end you have to evaluate the performance of the agent on 100 independent episodes on the environment and print out the average testing performance.

Make sure that your final submission is a notebook that can be run from beginning to end, and that you print out the performance of the agent at the end of the notebook.

In [13]:
import os
# !{os.sys.executable} -m pip install gymnasium
# !{os.sys.executable} -m pip install Pillow
# !{os.sys.executable} -m pip install ipython
# !{os.sys.executable} -m pip install pygame

from PIL import Image
from IPython.display import display

import tensorflow as tf
from collections import deque
import numpy as np
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import gymnasium as gym
from tensorflow import keras
from tensorflow.keras import layers

In [14]:
# Parameters
gamma = 0.99  # discount 
envname = "CartPole-v1"  # environment name
env=gym.make(envname, render_mode="rgb_array")

obssize = env.observation_space.low.size
actsize = env.action_space.n

## DQN (Deep Q Network)

In previous HWs, we have learned to use Tensorflow to build deep learning models. In this HW, we will apply deep learning as function approximations in reinforcement learning. 

Reference: DQN https://arxiv.org/abs/1312.5602


In tabular Q-learning, we maintain a table of state-action pairs $(s,a)$ and save one action value for each entry $Q(s,a),\forall (s,a)$. At each time step $t$, we are in state $s_t$, then we choose action based on $\epsilon-$greedy strategy. With prob $\epsilon$, choose action uniformly random; with prob $1-\epsilon$, choose action based on $$a_t = \arg\max_a Q(s_t,a)$$ 

We then get the instant reward $r_t$, update the Q-table using the following rule

$$Q(s_t,a_t) \leftarrow (1-\alpha)Q(s_t,a_t) + \alpha (r_t + \max_a \gamma Q(s_{t+1},a))$$

where $\alpha \in (0,1)$ is learning rate. The algorithm is shown to converge in tabular cases. However, in cases where we cannot keep a table for state and action, we need function approximation. Consider using neural network with parameter $\theta$, the network takes as input state $s$ and action $a$. (*or some features of state and action*). Let $Q_\theta(s,a)$ be the output of the network, which estimates the optimal Q-value function for state $s$ and action $a$.
$$Q_\theta(s,a) \approx Q^\ast(s,a)$$

In [15]:
def model_creator():
    model = keras.Sequential()
    model.add(layers.Dense(5,activation="relu"))
    # you should later modify this neural network
    model.add(layers.Dense(actsize,activation="linear")) # you should have one output for each possible action
    return model

We wish to train the neural network in order to find $\theta$ such that $Q_\theta(s,a)$ approximates $Q^*(s,a)$. As we discussed in the class, we can use observations of form $(s_i, a_i, r_i, s'_{i})$ (i.e., observing reward $r_i$ and new state $s'_{i}$ on taking action $a_i$ in state $s_i$) for training. Based on observations, we can use stochastic gradient descent to update $\theta$ in the direction that minimizes the loss function. Further, based on values $Q_\theta(s,a)$, we can choose the action based on $\epsilon$-greedy policy.

Formally let $d_i$ be the target for the $i$-th sample $(s_t,a_t,r_t,s_{t+1})$

$$d_i =  r_t +   \gamma \max_a Q_\theta(s_{t+1},a)$$

We can collect a batch of $N$ samples (this generalizes the per sample update $N=1$ discusssed in class), consider the loss fucntion,

$$J:=\frac{1}{N} \sum_{i=1}^N (Q_\theta(s_i,a_i) - d_i)^2$$

and update

$$
\theta \leftarrow \theta -\alpha \nabla_\theta J
$$

This procedure has been shown to be fairly unstable. In class, we discussed two techniques to stabilize the training process: target network and replay buffer.

**Replay Buffer**
Maintain a buffer $R$ to store trainsition tuples $(s_t,a_t,r_t,s_{t+1})$. When minimizing the loss, we sample batches from the replay buffer and compute gradients for update on these batches. In particular, in each update, we sample $N$ tuples $(s_t,a_t,r_t,s_{t+1})$ from buffer $R$ and then minimize the
loss 

$$\frac{1}{N} \sum_{i=1}^N (Q_\theta(s_i,a_i) -  (r_i + \gamma \max_a Q_\theta(s_i^\prime,a))^2$$

and update parameters.

**Target Network**
Maintain a target network in addition to the original principal network. The target network is just a copy of the original network but the parameters are not updated by gradients. The target network $\theta^{-}$ is copied from the principal network every $\tau$ time steps. Target network is used to compute the targets for update

$$d_i = \max_a r_t + \gamma Q_{\theta^{-}}(s_{i}^\prime,a)$$

the targets are used in the loss function to update the principal network parameters. This slowly updated target network ensures that the targets come from a relatively stationary distribution and hence stabilize learning.

In [16]:
batch_size = 32
# Model used for selecting actions (principal)
model = model_creator()
# Then create the target model. This will periodically be copied from the principal network 
model_target = model_creator()

model.build((batch_size,obssize,))
model_target.build((batch_size,obssize,))

- Complete the code below to learn an Agent using DQN. 
- You should tweak the Neural network appropriately to achieve a good reward (>100). Ideally you would want to have a reward close to 200.
- The reference paper performs updates every 4 actions. You can experiment with this parameter to speed up the learning
- You can experiment with other parameters as well, like learning rate, memory size, different exploration schemes (e.g., adaptive $\epsilon$ or strategic explorations with bonus rewards) and others.

- As we mentioned in class, there are multiple ways to improve the efficiency even further. OPTIONALLY you can experiment with these:
  - Prioritized Replay buffer.
  - Double DQN 
  -Dueling DQN architecture.

- In case you need to debug your code you can try printing relevant information as the training happens. For example although the performance might vary from iteration to iteration, the average Q values might increase overtime in a more smooth way. This is discussed in the refernece paper

- Create a plot of the running reward sampled throughout the training at the frequency of your choice at the end of the training
- OPTIONALLY you can create a plot for the average Q-values of the principal Q-network sampled at the frequency of your choice

- Ideally you want to learn with as few episodes as possible. However you will not be graded on sample efficiency in this homework. You encouraged to try to learn efficiently though.

- Note that the skeleton code includes the GradientTape construct to do the learning. Take a look [here](https://www.tensorflow.org/api_docs/python/tf/GradientTape) for an explanation of GradientTape. It allows for more flexibility than model.fit. Also it uses Adam (Adaptive Moment Estimation) for Stochastic Gradient Descent optimizer. 

In [31]:
import random

optimizer = keras.optimizers.Adam(learning_rate=0.0005)

# Our Experience Replay memory 
action_history = []
state_history = []
state_next_history = []
rewards_history = []
done_history = []
episode_reward_history = []

# Replay memory size
max_memory = 1000 # You can experiment with different sizes.

running_reward = 0
episode_count = 0
timestep_count = 0


# how often to train your model - this allows you to speed up learning
# by not performing in every iteration learning. See also refernece paper
# you can set this value to other values like 1 as well to learn every time 

update_after_actions = 4

# How often to update the target network
target_update_every = 1000
loss_function = keras.losses.Huber() # You can use the Huber loss function or the mean squared error function 
max_steps_per_episode = 1000

max_episodes = 500
# max_steps_per_episode = 1000
last_n_reward = 100


epsilon = 1 #means 100% probability of choosing random action
epsilon_decay_rate = 1/(max_episodes*0.9) # reduce the dependence on random actions over time
rng = np.random.default_rng() # creates a random number generator to be called later

for episode in range(max_episodes):
    #state = np.array(env.reset()) #their code
    state = env.reset()[0]
    state = np.array(state)
    print("state", state)
    
    episode_reward = 0

    for timestep in range(1, max_steps_per_episode):
        timestep_count += 1
        
        # exploration
        if rng.random() < epsilon:
            # Take random action
            action = np.random.choice(actsize)
        else:
            # Predict action Q-values
            # From environment state
            state_t = tf.convert_to_tensor(state)
            state_t = tf.expand_dims(state_tensor, 0)
            action_vals = model(state_t, training=False)
            # Choose the best action
            action = np.argmax(action_vals)

        # follow selected action
        # asdf = env.step(action)
        # print("env.step(action)", asdf, len(asdf))
        # state_next, reward, done, _ = env.step(action) #their code
        state_next, reward, done, _, _ = env.step(action) # my code
        state_next = np.array(state_next)
        episode_reward += reward

        # Save action/states and other information in replay buffer
        action_history.append(action)
        state_history.append(state)
        state_next_history.append(state_next)
        rewards_history.append(reward)
        done_history.append(done)
        # ...
        # ...

        state = state_next

        # Update every Xth frame to speed up (optional)
        # and if you have sufficient history
        if timestep_count % update_after_actions == 0 and len(action_history) > batch_size:
            
            #  Sample a set of batch_size memories from the history
            random_indices = random.sample(range(len(action_history)), batch_size)
            
            state_sample = [state_history[i] for i in random_indices]
            state_next_sample = [state_next_history[i] for i in random_indices]
            rewards_sample = [rewards_history[i] for i in random_indices]
            done_sample = [done_history[i] for i in random_indices]
            action_sample = [action_history[i] for i in random_indices]

            # Create for the sample states the targets (r+gamma * max Q(...) )
            Q_next_state = model_target.predict(state_next_sample)
            Q_targets = rewards_sample + gamma * np.max(Q_next_state)

            # If the episode was ended (done_sample value is 1)
            # you can penalize the Q value of the target by some value `penalty`
            Q_targets = Q_targets * (1 - done_sample) - penalty*done_sample


            # What actions are relevant and need updating
            relevant_actions = tf.one_hot(action_sample, actsize)
            # we will use Gradient tape to do a custom gradient 
            # in the `with` environment we will record a set of operations
            # and then we will take gradients with respect to the trainable parameters
            # in the neural network
            with tf.GradientTape() as tape:
                # Train the model on your action selecting network
                q_values = model(state_sample) 
                # We consider only the relevant actions
                Q_of_actions = tf.reduce_sum(tf.multiply(q_values, relevant_actions), axis=1)
                # Calculate loss between principal network and target network
                loss = loss_function(Q_targets, Q_of_actions)

            # Nudge the weights of the trainable variables towards 
            grads = tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(zip(grads, model.trainable_variables))

        if timestep_count % target_update_every == 0:
            # update the the target network with new weights
            model_target.set_weights(model.get_weights())
            # Log details
            template = "running reward: {:.2f} at episode {}, frame count {}, epsilon {}"
            print(template.format(running_reward, episode_count, frame_count,epsilon))
            
        # Don't let the memory grow beyond the limit
        if len(rewards_history) > max_memory:
            del rewards_history[:1]
            del state_history[:1]
            del state_next_history[:1]
            del action_history[:1]
            del done_history[:1]
        if done: break

    # reward of last 100
    episode_reward_history.append(episode_reward)
    if len(episode_reward_history) > last_n_reward: del episode_reward_history[:1]
    running_reward = np.mean(episode_reward_history)
    episode_count += 1
    
    epsilon = max(0, epsilon - epsilon_decay_rate)

    # If you want to stop your training once you achieve the reward you want you can
    # have an if statement here. Alternatively you can stop after a fixed number
    # of episodes.

state [-0.01197494  0.00132405 -0.03359745  0.02057897]
state [-0.00534606 -0.03792212  0.01268156  0.00066988]
state [-0.00361048 -0.01647903  0.04563836 -0.01182302]


ValueError: Exception encountered when calling Sequential.call().

[1mInvalid input shape for input Tensor("data:0", shape=(4,), dtype=float32). Expected shape (32, 4), but input has incompatible shape (4,)[0m

Arguments received by Sequential.call():
  • inputs=('tf.Tensor(shape=(4,), dtype=float32)', 'tf.Tensor(shape=(4,), dtype=float32)', 'tf.Tensor(shape=(4,), dtype=float32)', 'tf.Tensor(shape=(4,), dtype=float32)', 'tf.Tensor(shape=(4,), dtype=float32)', 'tf.Tensor(shape=(4,), dtype=float32)', 'tf.Tensor(shape=(4,), dtype=float32)', 'tf.Tensor(shape=(4,), dtype=float32)', 'tf.Tensor(shape=(4,), dtype=float32)', 'tf.Tensor(shape=(4,), dtype=float32)', 'tf.Tensor(shape=(4,), dtype=float32)', 'tf.Tensor(shape=(4,), dtype=float32)', 'tf.Tensor(shape=(4,), dtype=float32)', 'tf.Tensor(shape=(4,), dtype=float32)', 'tf.Tensor(shape=(4,), dtype=float32)', 'tf.Tensor(shape=(4,), dtype=float32)', 'tf.Tensor(shape=(4,), dtype=float32)', 'tf.Tensor(shape=(4,), dtype=float32)', 'tf.Tensor(sha

Evaluate the performance of the agent on 100 episodes on the environment and print out the average testing performance. Alternatively you can make sure the code above terminates with 100 episodes where there is no exploration at all (epsilon=0).

In [ ]:
# YOUR CODE Here